Bert-Base-Uncased


# 1 Pointwise BERT Finetuning

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tqdm import tqdm
import os
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer('He is rich')

{'input_ids': [101, 2002, 2003, 4138, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/triples.train.small.tar.gz
!tar -xf triples.train.small.tar.gz


In [ ]:
##
TPU = True
if TPU:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
else:
  pass

In [ ]:
def convert_data(data_df):    # Input: tokens(tokens encoded), segments, masks
  global tokenizer
  tokens, segments, masks, targets = [], [], [] ,[]
    
  for i in range(len(data_df)):
   
    targets.append(data_df["relevant"][i])

    que = tokenizer.encode(data_df["query"][i])
    doc = tokenizer.encode(data_df["passage"][i])
        
    doc.pop(0) 
        
        # Maximum Seq_len : cutting the length
    if len(que+doc) > SEQ_LEN:  #200 
      while len(que+doc) != SEQ_LEN:
        doc.pop(-1)  # Cut context
      doc.pop(-1)
      doc.append(102) #[SEP]
          
        # Segment
        # query, passage, padding
        # 00000000, 1111111, 0000000
    segment = [0]*len(que) + [1]*len(doc) + [0]*(SEQ_LEN-len(que)-len(doc))
        
        # Masking, Padding
    if len(que + doc) <= SEQ_LEN:
      mask = [1]*len(que+doc) + [0]*(SEQ_LEN-len(que+doc))
    else:
      mask = [1]*len(que+doc)   #No mask
          
    if len(que + doc) <= SEQ_LEN:
      while len(que+doc) != SEQ_LEN: #  padding
        doc.append(0)  #passage  padding
                              
    ids = que + doc # padding 
        
    tokens.append(ids) # [ids], [ids], ........
    segments.append(segment)
    masks.append(mask)
        
    # make numpy array 
  tokens = np.array(tokens)
  segments = np.array(segments)
  masks = np.array(masks)
  targets = np.array(targets)

  return [tokens, masks, segments], targets

def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df["query"] = data_df["query"].astype(str)     # String
  data_df["passage"] = data_df["passage"].astype(str)
  data_x, data_y = convert_data(data_df)

  return data_x, data_y   # [tokens,masks,segments], targets


In [ ]:
import pandas as pd 
train_pos = pd.read_csv("/content/triples.train.small.tsv",
                    sep = "\t", nrows = 800000, header = None, skiprows = 1, usecols = [0,1])  1600000
train_neg = pd.read_csv("/content/triples.train.small.tsv",
                    sep = "\t", nrows = 800000, header = None, skiprows = 1, usecols = [0,2]) 

train_pos.columns = ["query", "passage"]
train_neg.columns = ["query", "passage"]
train_pos["relevant"] = 1 # target label
train_neg["relevant"] = 0 

#train = train_pos.append(train_neg)   # List adding
train = pd.concat([train_pos, train_neg])
train.reset_index(inplace=True, drop=True)           # Resetting Index as Concat makes duplicated sets 00 11 22 33 44 
train

In [ ]:
import numpy as np
SEQ_LEN = 200 
BATCH_SIZE = 20         # Small for CPU, GPU        
EPOCHS=2 
LR=1.0e-5

train_x, train_y = load_data(train)        # Load Data

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa

def get_bert_finetuning_model():
  
  model = TFBertModel.from_pretrained("bert-base-uncased")
         
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')

  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])         # Putting input into Bert

  bert_outputs = bert_outputs[1]
  classification = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)  
  BertRanking_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], classification)
  opt = tfa.optimizers.RectifiedAdam(lr=LR, weight_decay=0.0025, warmup_proportion=0.05)
  BertRanking_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])     

  return BertRanking_model

##########################

path = "gdrive/My Drive/Colab Notebooks/BERT"
if TPU:
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
 
  with strategy.scope():
    BertRanking_model = get_bert_finetuning_model()   # Using TPU 
  BertRanking_model.fit(train_x, train_y, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, validation_split= 0.2) 
  BertRanking_model.save_weights(path+"/BERT_new.h5")

  # GPU 
else:
  BertRanking_model = get_bert_finetuning_model()
  BertRanking_model.fit(train_x, train_y, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, validation_split= 0.2)
  BertRanking_model.save_weights(path+"/BERT_new.h5")

In [ ]:
new_model = get_bert_finetuning_model()
new_model.load_weights(path+"/BERT_new.h5")
new_model.weights

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 37.9 gigabytes of available RAM

You are using a high-RAM runtime!


## Top 50 rank

In [ ]:
def get_bert_finetuning_model():
  
  model = TFBertModel.from_pretrained("bert-base-uncased")
         
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')

  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])         # Putting input into Bert

  bert_outputs = bert_outputs[1]
  classification = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)  # input for Sequential 
  BertRanking_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], classification)
  opt = tfa.optimizers.RectifiedAdam(lr=LR, weight_decay=0.0025, warmup_proportion=0.05)
  BertRanking_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])    

  return BertRanking_model

     |████████████████████████████████| 686kB 5.1MB/s 


In [ ]:
pointwise_model = get_bert_finetuning_model()
pointwise_model.load_weights("/content/drive/MyDrive/Colab Notebooks/BERT/BERT_new.h5")   
print("pointwise_model loaded")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


pointwise_model loaded


Convert Pointwise data for Prediction

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in range(len(data_df)):
        que = tokenizer.encode(data_df.iloc[i,2])
        doc = tokenizer.encode(data_df.iloc[i,3])
        
        doc.pop(0) 
        # Maximum Seq_len : cutting the length
        if len(que+doc) > SEQ_LEN:  #200 
          while len(que+doc) != SEQ_LEN:
            doc.pop(-1)  # Cut context
          doc.pop(-1)
          doc.append(102) #[SEP]
          
        # Segment
        # query, passage, padding
        # 00000000, 1111111, 0000000
        segment = [0]*len(que) + [1]*len(doc) + [0]*(SEQ_LEN-len(que)-len(doc))
        
        # Masking, Padding
        if len(que + doc) <= SEQ_LEN:
          mask = [1]*len(que+doc) + [0]*(SEQ_LEN-len(que+doc))
        else:
          mask = [1]*len(que+doc)   #No mask
          
        if len(que + doc) <= SEQ_LEN:
          while len(que+doc) != SEQ_LEN: #  padding
            doc.append(0)  #passage  padding
                              
        ids = que + doc # padding 
        
        tokens.append(ids) # [ids], [ids], ........
        segments.append(segment)
        masks.append(mask)
    tokens = np.array(tokens)
    segments = np.array(segments)
    masks = np.array(masks)
    

    return [tokens, masks, segments]

def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe.copy(deep=False)
    data_df["query"] = data_df["query"].astype(str).copy(deep=False)     # String
    data_df["passage"] = data_df["passage"].astype(str).copy(deep=False)
    data_x = predict_convert_data(data_df)
    return data_x

In [ ]:
import datetime
import csv
import pandas as pd 
                                                                         
data_all = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/BERT/all_dev3.csv", nrows = 100000, skiprows = 0,  sep="\t", header=None)    #sep=","
data_all.columns = ['q_id','p_id','query', 'passage']
unqiue_ids = data_all.q_id.unique()
data_all = data_all.to_numpy()
print(unqiue_ids)
print(len(unqiue_ids))

[    2  1215  1288  1576  2235  2798  2962  4696  4947  5925  6217  6791
  7968  8701  8714  8798  8854  9083  9454  9926 10157 10205 10264 10276
 10312 11006 11050 11133 11913 12741 12903 13397 14151 14947 14963 15039
 15063 15382 15441 15607 16559 16860 17110 17430 17586 17635 17848 17884
 18101 18759 18840 19457 19552 19940 20356 20432 20520 20597 20671 21185
 21603 21741 21765 21793 21861 21948 22231 22479 22670 22882 23223 23285
 24115 24441 24807 24979 25025 25036 25294 25344 25534 25603 26079 26207
 26334 26485 26664 26847 27618 27743 27932 28216 28352 28442 29089 29097
 29169 29416 29612 29921 30039 30188]
102


In [ ]:
def take_third(elem):
    return elem[2]

def from_data_to_array(data):
    data_ma = []
    for row in data:                                              
        data_ma.append([row[0], row[1], row[2], row[3]])
    return data_ma

for id in unqiue_ids:
    a = datetime.datetime.now()
    matrix = []
    data_ma = from_data_to_array(data[data[:, 0] == id])  #  First index : query_id
    df = pd.DataFrame(data_ma, columns = ['q_id','p_id','query', 'passage'])
    train_data = predict_load_data(df)
    
    preds = pointwise_model.predict(train_data)
    #print(preds)
    
    for i in range(0, len(data_ma)):
        matrix.append([data_ma[i][0], data_ma[i][1], np.float32(preds[i])])
    matrix = sorted(matrix, key=take_third, reverse=True)
    matrix = pd.DataFrame(matrix,columns=['q_id','p_id','score'])
    #print(matrix)

    matrix.to_csv("result{}.csv".format(id), index = False)

! zip pointwise_results.zip *.csv
print("ZIP done!!!!!")

from google.colab import files
files.download('pointwise_results.zip')

In [ ]:
upload = files.upload(pointwise_results.zip)

In [ ]:
ls -l 

# 2 Pairwise BERT Finetuning

In [ ]:
!tar -xzvf "/content/drive/MyDrive/Colab Notebooks/BERT/triples.train.small.tar.gz" -C "/content/"

triples.train.small.tsv


Duplicate triples data set by changing "relevant and non-relevant" column

In [ ]:
import pandas as pd 
trainpair_pos = pd.read_csv("/content/triples.train.small.tsv", sep = "\t", nrows =300000, header = None, skiprows = 1, usecols = [0,1,2]) 
trainpair_pos.columns = ["query", "passage_rel", "passage_non"]                 # 500000
trainpair_neg = trainpair_pos.copy()          #Copy 

trainpair_neg = trainpair_neg[["query", "passage_non", "passage_rel"]]
trainpair_neg.columns = ["query", "passage_rel", "passage_non"]          
trainpair_pos["relevant"] = 1
trainpair_neg["relevant"] = 0

trainpair = pd.concat([trainpair_pos, trainpair_neg])
trainpair.reset_index(inplace=True, drop=True)   # Resetting Index
trainpair

Pairwise converting

In [ ]:
def convert_data(data_df):    # Input: tokens(tokens encoded), segments, masks
  global tokenizer
  tokens, segments, masks, targets = [], [], [] ,[]
    
  for i in range(len(data_df)):    
    targets.append(data_df["relevant"][i])

    que = tokenizer.encode(data_df["query"][i])
    doc_1 = tokenizer.encode(data_df["passage_rel"][i])
    doc_2 = tokenizer.encode(data_df["passage_non"][i])
        
    doc_1.pop(0) 
    doc_2.pop(0) 
        
        # Maximum Seq_len : cutting the length
    if len(que+doc_1+doc_2) > SEQ_LEN:  #512
      while len(que+doc_1+doc_2) != SEQ_LEN:
        doc_2.pop(-1)   
      doc_2.pop(-1)
      doc_2.append(102) 
          
    # 1) Segment
    # query, passage, padding
    # 00000 11111 222222 000000
    segment = [0]*len(que) + [1]*len(doc_1) + [2]*len(doc_2) + [0]*(SEQ_LEN-len(que)-len(doc_1)-len(doc_2))
        
    # 2) Masking
    if len(que + doc_1 + doc_2) <= SEQ_LEN:
      mask = [1]*len(que+doc_1+doc_2) + [0]*(SEQ_LEN-len(que)-len(doc_1)-len(doc_2))       
    else:
      mask = [1]*len(que+doc_1+doc_2)   #No mask

    # 3) Padding for Token Embedding      
    if len(que + doc_1 + doc_2) <= SEQ_LEN:
      while len(que + doc_1 + doc_2) != SEQ_LEN: 
        doc_2.append(0)  #passage_non padding
                              
    ids = que + doc_1 + doc_2 # padding 
        
    tokens.append(ids) # [ids], [ids], 
    segments.append(segment)
    masks.append(mask)
        
    # make numpy array 
  tokens = np.array(tokens)
  segments = np.array(segments)
  masks = np.array(masks)
  targets = np.array(targets)
  return [tokens, masks, segments], targets

def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df["query"] = data_df["query"].astype(str)     # String
  data_df["passage_rel"] = data_df["passage_rel"].astype(str)
  data_df["passage_non"] = data_df["passage_non"].astype(str)
  data_x, data_y = convert_data(data_df)
  return data_x, data_y   # [tokens, masks, segments], targets  = (X. Y)

train_x, train_y = load_data(trainpair)        # Load Data

Pairwise training and save weights

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/BERT" 

if TPU:
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
 
  with strategy.scope():
    BertRanking_model = get_bert_finetuning_model()   # Using TPU 
  BertRanking_model.fit(train_x, train_y, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, validation_split= 0.2) 
  BertRanking_model.save_weights(path+"/BERT_pairwise_2.h5")
  print("saved")
  # GPU 
else:
  BertRanking_model = get_bert_finetuning_model()
  BertRanking_model.fit(train_x, train_y, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, validation_split= 0.2)
  BertRanking_model.save_weights(path+"/BERT_pairwise_2.h5")
  print("saved")

     |████████████████████████████████| 686kB 7.0MB/s 


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:2' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:2' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>]


48000/48000 [==============================] - ETA: 0s - loss: 0.0908 - accuracy: 0.9629

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:2' shape=(None, 512) dtype=int64>, <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>]


48000/48000 [==============================] - 5535s 113ms/step - loss: 0.0908 - accuracy: 0.9629 - val_loss: 0.0438 - val_accuracy: 0.9839
Epoch 2/2
48000/48000 [==============================] - 5435s 113ms/step - loss: 0.0308 - accuracy: 0.9880 - val_loss: 0.1199 - val_accuracy: 0.9588
saved


In [ ]:
pairwise_model = get_bert_finetuning_model()
pairwise_model.load_weights(path+"/BERT_pairwise.h5")
pairwise_model.weights

## Top 50 Re-rank

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/BERT" 
pairwise_model = get_bert_finetuning_model()
pairwise_model.load_weights(path+"/BERT_pairwise.h5")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/BERT/all_dev3.csv", nrows = 1000, skiprows = 0,  sep="\t", header=None)    #sep=","
data.columns = ['q_id','p_id','query', 'passage']
unqiue_ids = data.q_id.unique()
all_dev = data.to_numpy()
all_dev
#print(unqiue_ids)
#print(len(unqiue_ids))

array([[2, 2700543, ' Androgen receptor define',
        'Structure-function of the glucagon receptor family of G protein-coupled receptors: the glucagon, GIP, GLP-1, and GLP-2 receptors. Receptor Channels. 2002, 8 (3–4): 179–188.'],
       [2, 2726978, ' Androgen receptor define',
        'In pharmacology, an antagonist binds to the receptor cells and blocks or suppresses the normal response of the receptors. Thus, it is easy to see that while an agonist drug initiates a response from the body, an antagonist blocks the normal response of the cell receptor. An antagonist prevents a reaction.n pharmacology, an antagonist binds to the receptor cells and blocks or suppresses the normal response of the receptors. Thus, it is easy to see that while an agonist drug initiates a response from the body, an antagonist blocks the normal response of the cell receptor. An antagonist prevents a reaction.'],
       [2, 2726974, ' Androgen receptor define',
        'A drug antagonist is any drug that 

Convert Pairwise data for Prediction

In [ ]:
def convert_data(data_df):    # Input: tokens(tokens encoded), segments, masks
  global tokenizer
  tokens, segments, masks = [], [], []
    
  for i in range(len(data_df)):    
    que = tokenizer.encode(data_df["query"][i])
    doc_1 = tokenizer.encode(data_df["passage_rel"][i])
    doc_2 = tokenizer.encode(data_df["passage_non"][i])
        
    doc_1.pop(0)
    doc_2.pop(0) 
        
        # Maximum Seq_len : cutting the length
    if len(que+doc_1+doc_2) > SEQ_LEN:  #512
      while len(que+doc_1+doc_2) != SEQ_LEN:
        doc_2.pop(-1)  
      doc_2.pop(-1)
      doc_2.append(102) #[SEP]
          
    # 1) Segment
    # query, passage, padding
    # 00000 11111 222222 000000
    segment = [0]*len(que) + [1]*len(doc_1) + [2]*len(doc_2) + [0]*(SEQ_LEN-len(que)-len(doc_1)-len(doc_2))
        
    # 2) Masking
    if len(que + doc_1 + doc_2) <= SEQ_LEN:
      mask = [1]*len(que+doc_1+doc_2) + [0]*(SEQ_LEN-len(que)-len(doc_1)-len(doc_2))       
    else:
      mask = [1]*len(que+doc_1+doc_2)   #No mask

    # 3) Padding for Token Embedding      
    if len(que + doc_1 + doc_2) <= SEQ_LEN:
      while len(que + doc_1 + doc_2) != SEQ_LEN: 
        doc_2.append(0)  #passage_non padding
                              
    ids = que + doc_1 + doc_2 # padding 
        
    tokens.append(ids) # [ids], [ids], ........
    segments.append(segment)
    masks.append(mask)
        
    # make numpy array 
  tokens = np.array(tokens)
  segments = np.array(segments)
  masks = np.array(masks)
  return [tokens, masks, segments]

def predict_load_data_pair(pandas_dataframe):
  data_df = pandas_dataframe.copy(deep=False)
  data_df["query"] = data_df["query"].astype(str).copy(deep=False)     # String
  data_df["passage_rel"] = data_df["passage_rel"].astype(str).copy(deep=False)
  data_df["passage_non"] = data_df["passage_non"].astype(str).copy(deep=False)
  data_x= convert_data(data_df)

  return data_x  # [tokens, masks, segments], targets  = (X. Y)

Use Top 50 ranking result from Pointwise

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/BERT/pointwise_results.zip" -d "/content/"

In [ ]:
for i in unqiue_ids:                                          # 50 * 49
  data = pd.read_csv("/content/result{}.csv".format(i), nrows = 50, skiprows = 1,  sep=",", header=None)    #sep=","  
  data = data.to_numpy()
  top_50 = []        
  for row in data:
    #print(row[])                                       
    top_50.append([row[0], row[1], all_dev[all_dev[:, 0] == row[0], 2][0], all_dev[all_dev[:, 1] == row[1], 3]])
    #print(top_50)
    #print(top_50)
  df = pd.DataFrame(top_50, columns = ["q_id", "p_id", "query", "passage"])
  query = df["query"][0]
  #q_id =  df["q_id"][0]
  p_id =  df["p_id"]
  #print(p_id)

  score_pairwise_sum = []
  for p1 in df["passage"]:   
    probas_pairwise = [] # one proba for each p1 - p2 combination 
    pairwise_pair = []
    
    for p2 in df["passage"]: # 49
        
        if p1 == p2:
          pass # only compare to the k-1 different passages
        else:  # apply model - get proba that p1 is more relevant than p2
          pairwise_pair.append([query, p1, p2])
          
    df_pair = pd.DataFrame(pairwise_pair, columns = ["query", "passage_rel", "passage_non"])
    pd.set_option('display.max_rows', 10)
        
    train_data = predict_load_data_pair(df_pair)
    preds = pairwise_model.predict(train_data)
    score = float(sum(preds))
    #print(score)
    score_pairwise_sum.append(score)

  df["score"] = score_pairwise_sum
  df_pair_score = df[["q_id", "p_id", "score"]].copy()
  df_pair_score = df_pair_score.sort_values(by = "score", ascending = False)
  df_pair_score.to_csv("result{}.csv".format(i), index = False)
  print("Saved")

0     4339068
1      378374
2     1001873
3     6920058
4     7579825
       ...   
45    6196668
46    2066157
47    6681590
48    1001876
49    2948035
Name: p_id, Length: 50, dtype: int64
Saved


In [ ]:
! zip pairwise_results.zip *.csv
print("ZIP done!!!!!")

In [ ]:
from google.colab import files
files.download('pairwise_results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>